In [1]:
%matplotlib inline

import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import datetime
from geopy import distance

In [101]:
vehicle_states = pd.read_csv("resources/vehicle_states.csv", sep=';', parse_dates=['last_seen','created_at'], index_col=['vehicle_id','last_seen'])
vehicle_states = vehicle_states.loc['EZ10_G2-005',:]
# remove bad data
vehicle_states = vehicle_states[vehicle_states['battery'] <= 100]
# sort
vehicle_states.sort_index(inplace=True)

In [94]:
# Average speed when moving forwards
df = vehicle_states[vehicle_states['speed'] > 0.0]
df['speed_kmh'] = df['speed'] * 3.6
print(df['speed'].mean())
print(df['speed'].std())
df.describe()

1.076796322289162
0.5441641294817422


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,latitude,longitude,theta,speed,battery,speed_kmh
count,26311.000000,26311.000000,26311.000000,26311.000000,26311.000000,26311.000000,26311.000000
mean,343562.699441,52.482072,13.357095,-0.046569,1.076796,59.506784,3.876467
std,242888.213369,0.000263,0.000307,1.776448,0.544164,18.827940,1.958991
min,1495.000000,52.481237,13.356464,-3.141337,0.025390,18.000000,0.091403
25%,126971.500000,52.481861,13.356856,-2.170163,0.651125,41.000000,2.344048
50%,258616.000000,52.482136,13.357166,0.290134,1.138000,60.000000,4.096800
75%,579916.500000,52.482230,13.357323,1.003841,1.434446,76.000000,5.164004
max,785696.000000,52.482734,13.357940,3.140694,2.111000,98.000000,7.599600


In [96]:
# Average speed when moving backwards
df = vehicle_states[vehicle_states['speed'] < 0.0]
df['speed_kmh'] = df['speed'] * 3.6
print(df['speed'].mean())
print(df['speed'].std())
df.describe()

-1.079000022572055
0.5421018368934848


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,latitude,longitude,theta,speed,battery,speed_kmh
count,31618.000000,31618.000000,31618.000000,31618.000000,31618.000000,31618.000000,31618.000000
mean,316200.793567,52.482104,13.357126,-0.107587,-1.079000,57.380100,-3.884400
std,231082.297694,0.000265,0.000298,1.759506,0.542102,18.554418,1.951567
min,1497.000000,52.481238,13.356443,-3.140694,-2.137988,18.000000,-7.696758
25%,125981.250000,52.481908,13.356942,-2.216427,-1.458448,40.000000,-5.250412
50%,249997.500000,52.482164,13.357183,-0.293239,-1.110994,57.000000,-3.999578
75%,507259.750000,52.482280,13.357345,0.997298,-0.658177,71.000000,-2.369436
max,785833.000000,52.482734,13.357939,3.141147,-0.025371,94.000000,-0.091337


In [97]:
df = vehicle_states

df['lonlat'] = list(zip(df['longitude'],df['latitude']))
df['previous_doors'] = df['doors'].shift(1)
df['last_seen_t'] = df.index
df['previous_last_seen_t'] = df['last_seen_t'].shift(1)
df['tvalue'] = df.index

df = df[df['doors'] != df['previous_doors']]
df['previous_lonlat'] = df['lonlat'].shift(1)
df['delta'] = (df['tvalue']-df['previous_last_seen_t']).fillna(0)
df['distance'] = df.apply(lambda row: distance.distance(row['lonlat'], row['previous_lonlat']).meters, axis=1)
drive_from_stop_to_stop = df[(df['doors'] == 't') & (df['previous_doors'] == 'f') & (df['distance'] > 10)]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [98]:
drive_from_stop_to_stop.head()

,id,latitude,longitude,theta,speed,battery,doors,created_at,lonlat,previous_doors,last_seen_t,previous_last_seen_t,tvalue,previous_lonlat,delta,distance
last_seen,,,,,,,,,,,,,,,,
2018-02-13 14:38:35,1621,52.482484,13.357516,2.447807,0.0,65,t,2018-02-13 14:38:35.768720,"(13.3575163305, 52.4824835363)",f,2018-02-13 14:38:35,2018-02-13 14:38:34,2018-02-13 14:38:35,"(13.357212609500001, 52.4821630661)",00:00:01,48.314154
2018-02-13 14:46:25,2568,52.482164,13.357211,2.450251,0.0,65,t,2018-02-13 14:46:25.709298,"(13.357211420699999, 52.4821637266)",f,2018-02-13 14:46:25,2018-02-13 14:46:25,2018-02-13 14:46:25,"(13.3575163558, 52.4824835154)",00:00:00,48.354759
2018-02-14 14:44:42,12738,52.482484,13.357515,2.452881,0.0,62,t,2018-02-14 14:44:42.806793,"(13.357514813900002, 52.4824844042)",f,2018-02-14 14:44:42,2018-02-14 14:44:42,2018-02-14 14:44:42,"(13.3572160165, 52.4821613009)",00:00:00,48.143880
2018-02-14 14:47:59,13133,52.481704,13.356615,-0.660107,0.0,62,t,2018-02-14 14:47:59.315787,"(13.3566148071, 52.4817037595)",f,2018-02-14 14:47:59,2018-02-14 14:47:58,2018-02-14 14:47:59,"(13.357514788, 52.482484461599995)",00:00:01,130.637489
2018-02-14 14:50:07,13390,52.482169,13.357216,2.481750,0.0,61,t,2018-02-14 14:50:07.811680,"(13.3572163437, 52.482168647200005)",f,2018-02-14 14:50:07,2018-02-14 14:50:06,2018-02-14 14:50:07,"(13.3566146266, 52.4817037021)",00:00:01,83.476663


In [119]:
df = vehicle_states

df['lonlat'] = list(zip(df['longitude'],df['latitude']))
df['last_seen_t'] = df.index
df['previous_battery'] = df['battery'].shift(1)

df = df[df['battery'] != df['previous_battery']]
df['previous_last_seen_t'] = df['last_seen_t'].shift()
df['previous_lonlat'] = df['lonlat'].shift(1)
df['delta'] = (df['last_seen_t']-df['previous_last_seen_t']).fillna(0)
df['distance'] = df.apply(lambda row: distance.distance(row['lonlat'], row['previous_lonlat']).meters, axis=1)

df = df[df.apply(lambda row: row['delta'].seconds, axis=1) > 0]
df['discharge_per_minute'] = (df['battery'] - df['previous_battery']) / df.apply(lambda row: row['delta'].seconds / 60, axis=1)
df = df[df['discharge_per_minute'] < 0]
df.describe()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

,id,latitude,longitude,theta,speed,battery,previous_battery,delta,distance,discharge_per_minute
count,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306,306.000000,306.000000
mean,352011.480392,52.482133,13.357179,0.547398,-0.022128,58.362745,59.428105,0 days 02:17:01.741830,21.094724,-13.020411
std,230899.026461,0.000182,0.000200,1.816988,0.647864,20.348499,20.293819,0 days 10:34:40.168635,30.176266,24.676973
min,1603.000000,52.481298,13.356470,-2.986946,-2.026997,18.000000,19.000000,0 days 00:00:01,0.000034,-60.000000
25%,149085.500000,52.482155,13.357193,-0.671414,0.000000,38.250000,40.250000,0 days 00:02:59.250000,0.031096,-0.334730
50%,324808.500000,52.482168,13.357216,0.742891,0.000000,60.500000,61.500000,0 days 00:15:45.500000,2.182207,-0.063473
75%,565552.000000,52.482170,13.357218,2.473498,0.000000,73.750000,74.750000,0 days 00:48:01,37.819630,-0.020879
max,782418.000000,52.482558,13.357754,2.841913,1.999955,97.000000,98.000000,4 days 20:26:55,136.955720,-0.000730
